In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from datetime import datetime
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### ADNIMERGE + additional data & data checks
Long's code to double-check and expand on ADNIMERGE sheet, translated to python/pandas from R

## Step 1: load ADNIMERGE.csv as base sheet to build on

In [2]:
adnimerge=pd.read_csv("ADNIspreadsheets/ADNIMERGE_28May2024.csv",na_values="",dtype={'RID':int, 'VISCODE':str}, parse_dates=['EXAMDATE'],low_memory=False)
adnimerge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16421 entries, 0 to 16420
Columns: 116 entries, RID to update_stamp
dtypes: datetime64[ns](1), float64(88), int64(5), object(22)
memory usage: 14.5+ MB


In [3]:
adnimerge.rename(columns={"VISCODE":'VISCODE2','EXAMDATE':"EXAMDATE_MERGE",'COLPROT':'PHASE_MERGE'},inplace=True)
adnimerge.drop(columns=['update_stamp',"Month",'M','ORIGPROT'],inplace=True)

In [4]:
# remove one case with VISCODE2 == m0
adnimerge=adnimerge.loc[adnimerge['VISCODE2'] != 'm0']

In [5]:
# check that no viscodes are labeled "sc"
print(f"INFO: VISCODE2 == 'sc': {len(adnimerge.loc[adnimerge['VISCODE2'] =='sc'])}")

INFO: VISCODE2 == 'sc': 0


In [6]:
## check for duplicates
dupes=adnimerge[adnimerge.duplicated(subset=["RID", 'VISCODE2'],keep=False)]
if len(dupes) > 0:
    print('WARNING:delete duplicate RID,VISCODE2 sessions')
else:
    print('INFO:no RID/VISCODE2 duplicates')

INFO:no RID/VISCODE2 duplicates


In [7]:
### Some cases with different VISCODE2 but same examdate_merge
adnimerge[adnimerge.duplicated(subset=['RID','EXAMDATE_MERGE'],keep=False)]

,RID,PHASE_MERGE,PTID,SITE,VISCODE2,EXAMDATE_MERGE,DX_bl,AGE,PTGENDER,PTEDUCAT,...,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl
4060,4783,ADNI2,035_S_4783,35,m30,2014-03-13,AD,83.2,Male,20,...,2.32432,596.3,221.7,22.07,1.14345,NaN,1.4989,NaN,1.51129,18.0984
4062,4783,ADNI2,035_S_4783,35,m18,2014-03-13,AD,83.2,Male,20,...,2.32432,596.3,221.7,22.07,1.14345,NaN,1.4989,NaN,1.51129,18.0984
4979,1195,ADNIGO,941_S_1195,941,m48,2011-02-15,CN,76.9,Male,20,...,NaN,NaN,NaN,NaN,1.30789,NaN,NaN,NaN,3.98631,47.7377
7676,1195,ADNI1,941_S_1195,941,m42,2011-02-15,CN,76.9,Male,20,...,NaN,NaN,NaN,NaN,1.30789,NaN,NaN,NaN,3.98631,47.7377
10056,1088,ADNI2,128_S_1088,128,m78,2013-01-23,LMCI,87.9,Male,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.05065,72.4590
10057,1088,ADNI2,128_S_1088,128,m72,2013-01-23,LMCI,87.9,Male,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.05065,72.4590


In [8]:
## 4783,2014-03-13: m30 is wrong? no mri for date/viscode2 anyway (all data the same)
## 1195,2011-02-15: m48 has an mri (m48 row has more data)
## 1088,2013-01-23: ?? (m72 row has more data)

adnimerge=adnimerge.drop([adnimerge.loc[(adnimerge['PTID'] == '035_S_4783') & (adnimerge['VISCODE2'] == 'm30')].index.values[0],
                         adnimerge.loc[(adnimerge['PTID'] == '941_S_1195') & (adnimerge['VISCODE2'] == 'm42')].index.values[0],
                         adnimerge.loc[(adnimerge['PTID'] == '128_S_1088') & (adnimerge['VISCODE2'] == 'm78')].index.values[0]]).reset_index(drop=True)

In [9]:
# print out basic information
print(f"INFO:{len(adnimerge)} rows and {len(adnimerge.columns)} columns")
print(f"INFO:{len(adnimerge['RID'].unique())} unique subjects")

INFO:16417 rows and 112 columns
INFO:2430 unique subjects


#### Check key columns for missing / incorrect values

In [10]:
keycols=['DX','PTGENDER','PTEDUCAT','PTETHCAT','PTRACCAT','PTMARRY','AGE','CDRSB','APOE4','MMSE','LDELTOTAL']
for col in keycols:
    print("______________________________")
    print(f"{adnimerge[col].value_counts(dropna=False)}")
    print()
    

______________________________
DX
MCI         4989
NaN         4959
CN          4020
Dementia    2449
Name: count, dtype: int64

______________________________
PTGENDER
Male      8977
Female    7440
Name: count, dtype: int64

______________________________
PTEDUCAT
16    3917
18    3221
20    2343
12    1856
14    1704
13     795
19     745
17     730
15     649
10     139
8      128
11      86
9       39
7       31
6       30
4        4
Name: count, dtype: int64

______________________________
PTETHCAT
Not Hisp/Latino    15794
Hisp/Latino          550
Unknown               73
Name: count, dtype: int64

______________________________
PTRACCAT
White                15098
Black                  760
Asian                  302
More than one          186
Unknown                 35
Am Indian/Alaskan       28
Hawaiian/Other PI        8
Name: count, dtype: int64

______________________________
PTMARRY
Married          12521
Widowed           1872
Divorced          1414
Never married      549
Un

In [11]:
adnimerge.head()

,RID,PHASE_MERGE,PTID,SITE,VISCODE2,EXAMDATE_MERGE,DX_bl,AGE,PTGENDER,PTEDUCAT,...,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl
0,2,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,16,...,NaN,NaN,NaN,NaN,1.33615,NaN,NaN,NaN,0.000000,0.00000
1,3,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,18,...,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,0.000000,0.00000
2,3,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,18,...,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,0.498289,5.96721
3,3,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,18,...,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,0.999316,11.96720
4,3,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,18,...,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,1.998630,23.93440


## Step 2: Diagnosis: check data

In [12]:
dxdf=pd.read_csv("ADNIspreadsheets/DXSUM_PDXCONV_03Jun2024.csv")
dxdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13817 entries, 0 to 13816
Data columns (total 41 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PHASE                 13817 non-null  object 
 1   PTID                  13817 non-null  object 
 2   RID                   13817 non-null  int64  
 3   VISCODE               13817 non-null  object 
 4   VISCODE2              13524 non-null  object 
 5   EXAMDATE              13758 non-null  object 
 6   DIAGNOSIS             13789 non-null  float64
 7   DXNORM                3868 non-null   float64
 8   DXNODEP               3868 non-null   float64
 9   DXMCI                 3868 non-null   float64
 10  DXMDES                11344 non-null  object 
 11  DXMPTR1               8098 non-null   float64
 12  DXMPTR2               8087 non-null   float64
 13  DXMPTR3               8093 non-null   float64
 14  DXMPTR4               8097 non-null   float64
 15  DXMPTR5            

In [13]:
# Remove rows where "VISCODE2" is null, "VISCODE2" == uns1, or "DIAGNOSIS" is null
print(f"{len(dxdf.loc[pd.isnull(dxdf['DIAGNOSIS'])])} null diagnosis values.")
oldlen=len(dxdf)
dxdf = dxdf.loc[(pd.notnull(dxdf['VISCODE2'])) & (dxdf['VISCODE2'] != 'uns1') & (pd.notnull(dxdf['DIAGNOSIS']))]
newlen=len(dxdf)
print(f"Dropped {oldlen-newlen} rows with null VISCODE2, DIAGNOSIS, or VISCODE2=='uns1'")

28 null diagnosis values.
Dropped 319 rows with null VISCODE2, DIAGNOSIS, or VISCODE2=='uns1'


In [14]:
print("Check for duplicate RID/VISCODE2:")
dxdf[dxdf.duplicated(subset=['RID','VISCODE2'],keep=False)]

Check for duplicate RID/VISCODE2:


,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXODES,DXCONFID,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
4342,ADNIGO,031_S_0830,830,m60,m60,2011-09-22,3.0,NaN,NaN,NaN,...,NaN,NaN,958,21,2014-07-08,NaN,NaN,NaN,NaN,2014-07-08 19:01:25.0
4717,ADNI2,031_S_0830,830,v06,m60,2011-09-26,3.0,NaN,NaN,NaN,...,NaN,NaN,764,21,2011-09-26,2011-09-26,NaN,NaN,NaN,2013-03-14 19:16:45.0


In [15]:
## remove instance where RID=830 & VISCODE == 'm60'
dxdf=dxdf.drop([dxdf.loc[(dxdf['RID'] == 830) & (dxdf['VISCODE'] == 'm60')].index.values[0]])
    
print(f"Check: should now be 0 duplicates: {len(dxdf[dxdf.duplicated(subset=['RID','VISCODE2'],keep=False)])}")

Check: should now be 0 duplicates: 0


In [16]:
## bl and sc are essentially the same visit.
## replace sc with bl, check for duplicate cases again
dxdf['TMP_VISCODE2']=None
for index,row in dxdf.iterrows():
    if row['VISCODE2'] == 'sc':
        dxdf.at[index,'VISCODE2'] = "bl"
        dxdf.at[index,'TMP_VISCODE2'] = 'sc'
        
print(f"Check: number of duplicates with sc replaced: \
{len(dxdf[dxdf.duplicated(subset=['RID','VISCODE2'],keep=False)])}")

Check: number of duplicates with sc replaced: 2930


In [17]:
## To remove duplicates, keep any duplicate that was originally a "bl"
dxdf_formatted=dxdf.sort_values('TMP_VISCODE2', ascending=False)\
                    .drop_duplicates(['RID','VISCODE2'],keep='last')\
                    .reset_index(drop=True)

print(f"Check: number of duplicates: \
{len(dxdf_formatted[dxdf_formatted.duplicated(subset=['RID','VISCODE2'],keep=False)])}")

Check: number of duplicates: 0


In [18]:
## select columns to merge
dxdf_tomerge=dxdf_formatted[['RID','VISCODE2','DIAGNOSIS']]
dxdf_tomerge.rename(columns={'DIAGNOSIS':"DX_validate"},inplace=True)

In [19]:
## do merge
adnimergedx = adnimerge.merge(dxdf_tomerge,on=['RID','VISCODE2'], how='left')
print(f"Gained {len(adnimergedx.loc[(pd.isnull(adnimergedx['DX'])) & (pd.notnull(adnimergedx['DX_validate']))])} data points.")
print(f"Still have {len(adnimergedx.loc[(pd.isnull(adnimergedx['DX'])) & (pd.isnull(adnimergedx['DX_validate']))])} null values.")

Gained 32 data points.
Still have 4927 null values.


In [20]:
adnimergedx.value_counts(['DX','DX_validate'])

DX        DX_validate
MCI       2.0            4989
CN        1.0            4020
Dementia  3.0            2449
Name: count, dtype: int64

In [21]:
## Extrapolate info from DX_validate to DX and make both float
for index,row in adnimergedx.iterrows():
    if pd.isnull(row['DX']) and pd.notnull(row['DX_validate']):
        if row['DX_validate'] == 1.0:
            dxval=1.0
        elif row['DX_validate'] == 2.0: 
            dxval =2.0
        elif row['DX_validate'] == 3.0: 
            dxval = 3.0
        else:
            print(f"{row['DX_validate']} invalid")
    elif pd.notnull(row['DX']):
        if row['DX'] == 'CN':
            dxval=1.0
        elif row['DX'] == 'MCI':
            dxval=2.0
        elif row['DX'] == 'Dementia':
            dxval=3.0
    else:
        dxval=None
    
    adnimergedx.at[index,'DX'] = dxval

print(adnimergedx.value_counts(['DX','DX_validate']))

DX   DX_validate
2.0  2.0            5002
1.0  1.0            4032
3.0  3.0            2456
Name: count, dtype: int64


In [22]:
## Check for any data points that are different both columns
nomatch=len(adnimergedx.loc[(pd.notnull(adnimergedx['DX'])) & (pd.notnull(adnimergedx["DX_validate"]))
& (adnimergedx['DX'] != adnimergedx['DX_validate'])])  ## if both are not null and value does not match

## If 0 columns different, drop validate column
if nomatch == 0:
    print('All values the same between original column and validate column, dropping validate column.')
    adnimergedx.drop(columns=['DX_validate'],inplace=True)
else:
    print('Some values different between original column and validate column, keeping validate column.')

adnimergedx.info()

All values the same between original column and validate column, dropping validate column.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 112 entries, RID to Month_bl
dtypes: datetime64[ns](1), float64(88), int64(3), object(20)
memory usage: 14.0+ MB


In [23]:
# If visit is baseline and DX is still null, use value from DX_bl column
print(f"{len(adnimergedx.loc[(pd.isnull(adnimergedx['DX'])) & (adnimergedx['VISCODE2'] =='bl')])} cases are baseline and have no DX value--use DX_bl value")
## at the moment, 2 cases and both are EMCI at bl. Use value 2.0
adnimergedx.loc[(pd.isnull(adnimergedx['DX'])) & (adnimergedx['VISCODE2'] =='bl'),'DX'] = 2.0

2 cases are baseline and have no DX value--use DX_bl value


In [24]:
print(adnimergedx.value_counts(['DX'],dropna=False))

DX 
2.0    5004
NaN    4925
1.0    4032
3.0    2456
Name: count, dtype: int64


## Step 3: Demographics: check data, add Birthdate

In [25]:
demog=pd.read_csv("./ADNIspreadsheets/PTDEMOG_ADNI1GO23.csv")
demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857 entries, 0 to 4856
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         4857 non-null   object 
 1   ID            4857 non-null   int64  
 2   PTID          4856 non-null   object 
 3   RID           4857 non-null   int64  
 4   SITEID        4857 non-null   int64  
 5   VISCODE       4857 non-null   object 
 6   VISCODE2      4855 non-null   object 
 7   VISDATE       4854 non-null   object 
 8   USERDATE      4857 non-null   object 
 9   USERDATE2     1699 non-null   object 
 10  PTSOURCE      4855 non-null   float64
 11  PTGENDER      4736 non-null   float64
 12  PTDOB         4679 non-null   object 
 13  PTDOBYY       4679 non-null   float64
 14  PTHAND        4723 non-null   float64
 15  PTMARRY       4850 non-null   float64
 16  PTEDUCAT      4722 non-null   float64
 17  PTWORKHS      2537 non-null   float64
 18  PTNOTRT       4837 non-null 

#### Missing demo data in adnimerge

In [26]:
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['PTMARRY'])])} null PTMARRY")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['PTGENDER'])])} null PTGENDER")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['PTEDUCAT'])])} null PTEDUCAT")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['PTETHCAT'])])} null PTETHCAT")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['PTRACCAT'])])} null PTRACCAT")
print()
print('Any null AGE values can be filled by birthdate addition:')
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['AGE'])])} null AGE")

1 null PTMARRY
0 null PTGENDER
0 null PTEDUCAT
0 null PTETHCAT
0 null PTRACCAT

Any null AGE values can be filled by birthdate addition:
9 null AGE


In [27]:
## check demog for data for the one subject missing PTMARRY:
demog.loc[demog['RID'] == adnimerge.loc[pd.isnull(adnimerge['PTMARRY'])]['RID'].values[0]]['PTMARRY'].values[0]

nan

#### Add birthdate to adnimerge

In [28]:
## Birthdate related cols only
demog2=demog[['RID','PTDOB','PTDOBYY']]
demog2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857 entries, 0 to 4856
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RID      4857 non-null   int64  
 1   PTDOB    4679 non-null   object 
 2   PTDOBYY  4679 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 114.0+ KB


In [29]:
# assume birth year later than 1974 to be invalid--make sure there are none
print(f"Check there are 0 birth years later than 1974: {len(demog2.loc[demog2['PTDOBYY'] >= 1974])}")

Check there are 0 birth years later than 1974: 0


In [30]:
## remove null values:
print(f"Removing {len(demog2.loc[pd.isnull(demog2['PTDOB'])])} null PTDOB values from demog sheet.")
demog3 = demog2.loc[(pd.notnull(demog2['PTDOBYY'])) & (pd.notnull(demog2['PTDOB']))].reset_index(drop=True)

Removing 178 null PTDOB values from demog sheet.


In [31]:
## Confirm: all dates the same for each RID
demosubs=demog3['RID'].unique()
for sub in demosubs:
    match=demog3.loc[demog3['RID'] == sub]
    alldobs=match['PTDOB'].values.tolist()
    if len(alldobs) > 1:
        # print(alldobs)
        setdob=set(alldobs)
        if len(setdob) > 1: 
            print('more than one')

In [32]:
## Drop duplicate RIDs
demog4 = demog3.drop_duplicates(subset='RID',keep='first').reset_index(drop=True)
demog4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4079 entries, 0 to 4078
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RID      4079 non-null   int64  
 1   PTDOB    4079 non-null   object 
 2   PTDOBYY  4079 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 95.7+ KB


In [33]:
## Add YYYY-MM-DD column to demog df
demog4['PTBIRTHDATE']=None
for index,row in demog4.iterrows():
    doblist=row['PTDOB'].split('/')
    newdob=doblist[1] + "-" + doblist[0]  + "-01"
    demog4.at[index,'PTBIRTHDATE'] = newdob
    
demog4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4079 entries, 0 to 4078
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   RID          4079 non-null   int64  
 1   PTDOB        4079 non-null   object 
 2   PTDOBYY      4079 non-null   float64
 3   PTBIRTHDATE  4079 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 127.6+ KB


In [34]:
demog4.drop(columns=['PTDOB'],inplace=True)

In [35]:
## Are all subs in adnimerge in demog sheet?
adnisubs=adnimerge['RID'].unique()
print(len(adnisubs))
count=0
for sub in adnisubs: 
    if sub in demosubs:
        count +=1
print(count)

2430
2430


In [36]:
## Add birthdates to ADNIMERGE df
adnimergebd=adnimergedx.merge(demog4,on=['RID'],how='left',validate="m:1")
## check for any nulls
len(adnimergebd.loc[pd.isnull(adnimergebd['PTBIRTHDATE'])])

0

In [37]:
## Convert PTBIRTHDATE to a datetime dtype
adnimergebd['PTBIRTHDATE']= pd.to_datetime(adnimergebd['PTBIRTHDATE'],format="%Y-%m-%d")
## fill in Age from examdate and PTBIRTHDATE
adnimergebd['AGE'] = (adnimergebd['EXAMDATE_MERGE'] - adnimergebd['PTBIRTHDATE']) / np.timedelta64(365, 'D')
## Check that all AGE values filled in
adnimergebd.loc[pd.isnull(adnimergebd['AGE'])]

,RID,PHASE_MERGE,PTID,SITE,VISCODE2,EXAMDATE_MERGE,DX_bl,AGE,PTGENDER,PTEDUCAT,...,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,PTDOBYY,PTBIRTHDATE


## Step 4: CDR: check data

In [38]:
cdr=pd.read_csv("ADNIspreadsheets/Cognition/CDR_24May2024.csv")
cdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13289 entries, 0 to 13288
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PHASE                 13289 non-null  object 
 1   PTID                  13289 non-null  object 
 2   RID                   13289 non-null  int64  
 3   VISCODE               13289 non-null  object 
 4   VISCODE2              13055 non-null  object 
 5   VISDATE               13281 non-null  object 
 6   CDDATE                1443 non-null   object 
 7   CDSOURCE              13139 non-null  float64
 8   CDVERSION             8323 non-null   float64
 9   SPID                  3200 non-null   float64
 10  CDMEMORY              13210 non-null  float64
 11  CDORIENT              13210 non-null  float64
 12  CDJUDGE               13210 non-null  float64
 13  CDCOMMUN              13209 non-null  float64
 14  CDHOME                13209 non-null  float64
 15  CDCARE             

In [39]:
## drop rows with no VISCODE2 and the one where VISCODE2 == 'uns1'
cdr2 = cdr.loc[(pd.notnull(cdr['VISCODE2'])) & (cdr['VISCODE2'] != 'uns1')]
len(cdr2)

13054

In [40]:
## Check for duplicates: 
cdr2[cdr2.duplicated(subset=['RID','VISCODE2'],keep=False)]

,PHASE,PTID,RID,VISCODE,VISCODE2,VISDATE,CDDATE,CDSOURCE,CDVERSION,SPID,...,CDGLOBAL,CDRSB,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
4259,ADNIGO,031_S_0830,830,m60,m60,NaN,NaN,1.0,NaN,NaN,...,0.5,4.0,1232,21,2014-07-08,NaN,NaN,NaN,NaN,2014-07-08 19:00:56.0
5107,ADNI2,031_S_0830,830,v06,m60,2011-09-22,NaN,1.0,1.0,NaN,...,0.5,4.0,630,21,2011-09-23,NaN,NaN,NaN,NaN,2012-11-16 12:23:27.0


In [41]:
# drop duplicate where RID == 830 and VISCODE==m60
cdr2=cdr2.drop([cdr2.loc[(cdr2['RID'] == 830) & (cdr2['VISCODE'] == 'm60')].index.values[0]])

In [42]:
## Double-check ADNI's CDRSB column by recalculating

## Replace the '-1' value with NaN
cdr2.replace({'CDMEMORY': -1.0,'CDORIENT': -1.0,'CDJUDGE': -1.0,'CDCOMMUN': -1.0,'CDHOME': -1.0,'CDCARE': -1.0}, None,inplace=True)
## Add up values for new columns
cdr2['CDRSB_validate'] = cdr2['CDMEMORY'] + cdr2['CDORIENT'] + cdr2['CDJUDGE'] + cdr2['CDCOMMUN'] + cdr2['CDHOME'] + cdr2['CDCARE']
## Compare ADNI's CDRSB with new calculation
cdr2.value_counts(['CDRSB_validate','CDRSB'],dropna=False)

CDRSB_validate  CDRSB
0.0             0.0      4164
0.5             0.5      1633
1.0             1.0      1368
1.5             1.5      1033
2.0             2.0       784
2.5             2.5       619
3.0             3.0       484
3.5             3.5       426
4.5             4.5       364
4.0             4.0       347
5.0             5.0       346
6.0             6.0       267
NaN             NaN       195
7.0             7.0       172
5.5             5.5       155
8.0             8.0       133
10.0            10.0      104
9.0             9.0        99
11.0            11.0       73
12.0            12.0       62
13.0            13.0       41
6.5             6.5        34
14.0            14.0       33
16.0            16.0       30
17.0            17.0       24
15.0            15.0       22
18.0            18.0       22
7.5             7.5        10
9.5             9.5         5
8.5             8.5         2
12.5            12.5        1
10.5            10.5        1
Name: count, dtype

In [43]:
## ADNI's calculated CDRSB is correct in CDR spreadsheet
cdrtomerge=cdr2[['RID','VISCODE2','CDRSB_validate']]

In [44]:
adnimerge_cdr = adnimergebd.merge(cdrtomerge,on=['RID','VISCODE2'],how='left')
adnimerge_cdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 115 entries, RID to CDRSB_validate
dtypes: datetime64[ns](2), float64(89), int64(3), object(21)
memory usage: 14.4+ MB


In [45]:
## Check that new values are the same as values in ADNIMERGE
adnimerge_cdr.value_counts(['CDRSB','CDRSB_validate'])

CDRSB  CDRSB_validate
0.0    0.0               2959
0.5    0.5               1177
1.0    1.0                929
1.5    1.5                687
2.0    2.0                535
2.5    2.5                432
3.0    3.0                371
3.5    3.5                312
4.5    4.5                274
5.0    5.0                273
4.0    4.0                252
6.0    6.0                215
7.0    7.0                136
5.5    5.5                122
8.0    8.0                108
10.0   10.0                99
9.0    9.0                 89
11.0   11.0                73
12.0   12.0                59
13.0   13.0                40
14.0   14.0                33
16.0   16.0                30
6.5    6.5                 26
17.0   17.0                24
15.0   15.0                22
18.0   18.0                22
7.5    7.5                  8
9.5    9.5                  5
8.5    8.5                  2
12.5   12.5                 1
10.5   10.5                 1
Name: count, dtype: int64

In [46]:
print(f"Gained {len(adnimerge_cdr.loc[(pd.isnull(adnimerge_cdr['CDRSB'])) & (pd.notnull(adnimerge_cdr['CDRSB_validate']))])} data points.")
print(f"Still have {len(adnimerge_cdr.loc[(pd.isnull(adnimerge_cdr['CDRSB'])) & (pd.isnull(adnimerge_cdr['CDRSB_validate']))])} null values.")

Gained 2 data points.
Still have 4668 null values.


In [47]:
## Add any new CDRSB_validate values to CDRSB column
new=0
for index,row in adnimerge_cdr.iterrows():
    if pd.isnull(row['CDRSB']) and pd.notnull(row['CDRSB_validate']):
        adnimerge_cdr.at[index,'CDRSB'] = row['CDRSB_validate']
        new +=1
        
print(f"Extrapolated {new} data points to original column")
print()

## Check for any data points that are different both columns
nomatch=len(adnimerge_cdr.loc[(pd.notnull(adnimerge_cdr['CDRSB'])) & (pd.notnull(adnimerge_cdr["CDRSB_validate"]))
& (adnimerge_cdr['CDRSB'] != adnimerge_cdr['CDRSB_validate'])])  ## if both are not null and value does not match


## If 0 columns different, drop validate column
if nomatch == 0:
    print('All values the same between original column and validate column, dropping validate column.')
    adnimerge_cdr.drop(columns=['CDRSB_validate'],inplace=True)
else:
    print('Some values different between original column and validate column, keeping validate column.')

print()
print(adnimerge_cdr.info())

Extrapolated 2 data points to original column

All values the same between original column and validate column, dropping validate column.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 114 entries, RID to PTBIRTHDATE
dtypes: datetime64[ns](2), float64(89), int64(3), object(20)
memory usage: 14.3+ MB
None


## Step 5: Amyloid Status: add data

In [48]:
amy=pd.read_csv("ADNIspreadsheets/PET/UCBERKELEY_AMY_6MM_29May2024.csv")
amy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805 entries, 0 to 3804
Columns: 343 entries, LONIUID to update_stamp
dtypes: float64(331), int64(3), object(9)
memory usage: 10.0+ MB


In [49]:
print(len(amy.loc[pd.notnull(amy['VISCODE'])]))

3805


#### Fill in VISCODE2 using registry

In [50]:
reg = pd.read_csv("ADNIspreadsheets/PET/REGISTRY_28May2024.csv")
regsm=reg[['PTID','VISCODE','VISCODE2']]

In [51]:
amyvis = amy.merge(regsm,on=['PTID','VISCODE'],how='left')
print(f"{len(amyvis.loc[pd.isnull(amyvis['VISCODE2'])])} null VISCODE2 after merge")

1 null VISCODE2 after merge


In [52]:
## drop rows with no VISCODE2
amyvis2 = amyvis.loc[(pd.notnull(amyvis['VISCODE2'])) & (amyvis['VISCODE2'] != 'uns1')]
len(amyvis2)

3927

In [53]:
## Check for duplicates: 
print(f"There are {len(amyvis2[amyvis2.duplicated(subset=['RID','VISCODE2'],keep=False)])} duplicates")
amyvis3=amyvis2.drop_duplicates(subset=['RID','VISCODE2'],keep='first')
print(f"Now have {len(amyvis3)} data points.")

There are 246 duplicates
Now have 3804 data points.


#### merge amyloid stats, suvr, centiloids

In [54]:
## double-check which columns to merge
amysm=amyvis3[['RID','VISCODE2','AMYLOID_STATUS','TRACER','SUMMARY_SUVR','CENTILOIDS']]

In [55]:
merge_amystat=adnimerge_cdr.merge(amysm,on=['RID','VISCODE2'],how='left')
print(f"Filled in {len(merge_amystat.loc[pd.notnull(merge_amystat['AMYLOID_STATUS'])])} values")

Filled in 3650 values


## Step 6: APOE: check data

In [56]:
apoe=pd.read_csv("ADNIspreadsheets/Biospecimen/APOERES_30May2024.csv")
apoe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2737 entries, 0 to 2736
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Phase         2737 non-null   object 
 1   ID            1159 non-null   float64
 2   PTID          2737 non-null   object 
 3   RID           2737 non-null   int64  
 4   SITEID        1159 non-null   float64
 5   VISCODE       1159 non-null   object 
 6   USERDATE      1159 non-null   object 
 7   USERDATE2     0 non-null      float64
 8   APTESTDT      1159 non-null   object 
 9   APGEN1        2737 non-null   int64  
 10  APGEN2        2737 non-null   int64  
 11  APVOLUME      1159 non-null   float64
 12  APRECEIVE     1159 non-null   float64
 13  APAMBTEMP     1159 non-null   float64
 14  APRESAMP      1159 non-null   float64
 15  APUSABLE      1159 non-null   float64
 16  update_stamp  2737 non-null   object 
dtypes: float64(8), int64(3), object(6)
memory usage: 363.6+ KB


In [57]:
## APGEN1 and APGEN2 all have values
print(apoe['APGEN1'].value_counts(dropna=False))
print(apoe['APGEN2'].value_counts(dropna=False))

APGEN1
3    2224
2     267
4     246
Name: count, dtype: int64
APGEN2
3    1501
4    1229
2       7
Name: count, dtype: int64


In [58]:
# compute genotype and APOE4 status

apoe['APOE_genotype'] = None
apoe['APOE4_validate'] = None
for index,row in apoe.iterrows():
    val=f"{row['APGEN1']};{row['APGEN2']}"
    # print(val)
    if val=="3;2":
        val="2;3"
    elif val == "4;2":
        val="2;4"
    elif val == "4;3":
        val="3;4"
    apoe.at[index,'APOE_genotype'] = val

    if "4" in val:
        if val == "4;4":
            num=2
        else:
            num=1
    else:
        num=0

    apoe.at[index,'APOE4_validate'] = num

print(apoe['APOE_genotype'].value_counts())
print(apoe['APOE4_validate'].value_counts())

APOE_genotype
3;3    1300
3;4     924
4;4     246
2;3     201
2;4      59
2;2       7
Name: count, dtype: int64
APOE4_validate
0    1508
1     983
2     246
Name: count, dtype: int64


In [59]:
## check for duplicates
print(len(apoe[apoe.duplicated(subset=['RID'],keep=False)]))

0


In [60]:
## select only columns to merge
apoesm=apoe[['RID','APOE_genotype','APOE4_validate']]

In [61]:
## merge
adnimerge_apoe=merge_amystat.merge(apoesm,on=['RID'],how='left')
len(adnimerge_apoe.loc[pd.isnull(adnimerge_apoe['APOE_genotype'])])

print(f"Gained {len(adnimerge_apoe.loc[(pd.isnull(adnimerge_apoe['APOE4'])) & (pd.notnull(adnimerge_apoe['APOE4_validate']))])} data points.")
print(f"Still have {len(adnimerge_apoe.loc[(pd.isnull(adnimerge_apoe['APOE4'])) & (pd.isnull(adnimerge_apoe['APOE4_validate']))])} null values.")

Gained 317 data points.
Still have 48 null values.


In [62]:
## Add any new _validate values to original column
new=0
for index,row in adnimerge_apoe.iterrows():
    if pd.isnull(row['APOE4']) and pd.notnull(row['APOE4_validate']):
        adnimerge_apoe.at[index,'APOE4'] = row['APOE4_validate']
        new +=1
        
print(f"Extrapolated {new} data points to original column")
print()

## Check for any data points that are different both columns
nomatch=len(adnimerge_apoe.loc[(pd.notnull(adnimerge_apoe['APOE4'])) & (pd.notnull(adnimerge_apoe["APOE4_validate"]))
& (adnimerge_apoe['APOE4'] != adnimerge_apoe['APOE4_validate'])])  ## if both are not null and value does not match


## If 0 columns different, drop validate column
if nomatch == 0:
    print('All values the same between original column and validate column, dropping validate column.')
    adnimerge_apoe.drop(columns=['APOE4_validate'],inplace=True)
else:
    print('Some values different between original column and validate column, keeping validate column.')

print()
print(adnimerge_apoe.info())

Extrapolated 317 data points to original column

Some values different between original column and validate column, keeping validate column.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 120 entries, RID to APOE4_validate
dtypes: datetime64[ns](2), float64(92), int64(3), object(23)
memory usage: 15.0+ MB
None


In [63]:
## Examine values that don't match up
dontmatch=adnimerge_apoe.loc[(pd.notnull(adnimerge_apoe['APOE4'])) & (adnimerge_apoe['APOE4'] != adnimerge_apoe['APOE4_validate'])]
dontmatchsm=dontmatch[['RID','VISCODE2','EXAMDATE_MERGE','APOE4','APOE4_validate']]
dontmatchsm.sort_values('RID')

,RID,VISCODE2,EXAMDATE_MERGE,APOE4,APOE4_validate
1636,777,bl,2006-09-21,0.0,1
14522,6066,bl,2017-09-07,0.0,1
14523,6066,m12,2018-08-28,0.0,1
14524,6066,m24,2019-11-06,0.0,1
15097,6436,bl,2018-07-25,1.0,0
15098,6436,m12,2019-08-08,1.0,0
15099,6436,m24,2020-10-07,1.0,0
15716,6436,m42,2021-12-06,1.0,0
15296,6564,bl,2018-10-02,2.0,1
16306,6564,m48,2022-07-19,2.0,1


In [64]:
dontmatch['RID'].value_counts()

RID
6436    4
6066    3
6564    3
777     1
Name: count, dtype: int64

In [65]:
for sub in dontmatch['RID'].unique():
    inorig=apoe.loc[apoe['RID'] == sub]
    print(f"Subject {sub} {inorig['APGEN1'].values},{inorig['APGEN2'].values}")

Subject 777 [3],[4]
Subject 6066 [2],[4]
Subject 6436 [3],[3]
Subject 6564 [3],[4]


## Step 7: Plasma: add data

In [66]:
## new data, no existing Plasma info in adnimerge
## data only exists for ADNI 1 & 2
plasma=pd.read_csv("ADNIspreadsheets/Biospecimen/ADNI_BLENNOWPLASMANFLLONG_10_03_18_30May2024.csv")
plasma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RID           3762 non-null   int64  
 1   VISCODE       3762 non-null   object 
 2   VISCODE2      3762 non-null   object 
 3   USERDATE      3762 non-null   object 
 4   EXAMDATE      3762 non-null   object 
 5   RECNO         3762 non-null   int64  
 6   DRAW_DATE     3762 non-null   object 
 7   DRAW_TIME     3744 non-null   object 
 8   PLASMA_NFL    3761 non-null   float64
 9   COMMENT       1 non-null      object 
 10  update_stamp  3762 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 323.4+ KB


In [67]:
# manually change VISCODE2 of some subjects
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 548), 'VISCODE2'] = 'm48'
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 1256), 'VISCODE2'] = 'm84'
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 4455), 'VISCODE2'] = 'm48'
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 4499), 'VISCODE2'] = 'm48'
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 4660), 'VISCODE2'] = 'm12'
plasma.loc[(plasma['VISCODE'] == 'nv') & (plasma['RID'] == 4993), 'VISCODE2'] = 'bl'
plasma.loc[(plasma['VISCODE'] == 'sc') & (plasma['RID'] == 113), 'VISCODE2'] = 'm60'
plasma.loc[(plasma['VISCODE'] == 'v02') & (plasma['RID'] == 4020), 'VISCODE2'] = 'bl'

In [68]:
## keep only rows with viscode2 and plasma_nfl data, where viscode2 != 'nv'
plasma2=plasma.loc[(pd.notnull(plasma['VISCODE2'])) & (pd.notnull(plasma['PLASMA_NFL'])) & (plasma['VISCODE2'] != 'nv')]
plasma2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3759 entries, 0 to 3761
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RID           3759 non-null   int64  
 1   VISCODE       3759 non-null   object 
 2   VISCODE2      3759 non-null   object 
 3   USERDATE      3759 non-null   object 
 4   EXAMDATE      3759 non-null   object 
 5   RECNO         3759 non-null   int64  
 6   DRAW_DATE     3759 non-null   object 
 7   DRAW_TIME     3741 non-null   object 
 8   PLASMA_NFL    3759 non-null   float64
 9   COMMENT       0 non-null      object 
 10  update_stamp  3759 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 352.4+ KB


In [69]:
print(f"there are {len(plasma2[plasma2.duplicated(subset=['RID','VISCODE2'],keep=False)])} duplicates")

there are 108 duplicates


In [70]:
plasma2.drop_duplicates(subset=['RID','VISCODE2'],keep='first',inplace=True)

In [71]:
plasmasm=plasma2[['RID','VISCODE2', 'PLASMA_NFL','DRAW_DATE']]
plasmasm.rename(columns={'DRAW_DATE':'PLASMANFL_DRAWDATE'},inplace=True)

In [72]:
merge_plasma = adnimerge_apoe.merge(plasmasm,on=['RID','VISCODE2'],how='left')
print(f"Filled in {len(merge_plasma.loc[pd.notnull(merge_plasma['PLASMA_NFL'])])} values")

Filled in 3701 values


## Step 8: PTau: add data

In [73]:
### new data
ptau=pd.read_csv("ADNIspreadsheets/Biospecimen/UGOTPTAU181_06_18_20_30May2024.csv")
ptau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3758 entries, 0 to 3757
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RID            3758 non-null   int64  
 1   VISCODE        3758 non-null   object 
 2   VISCODE2       3758 non-null   object 
 3   EXAMDATE       3758 non-null   object 
 4   VID            3758 non-null   int64  
 5   PLASMAPTAU181  3758 non-null   float64
 6   COMMENT        6 non-null      object 
 7   update_stamp   3758 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 235.0+ KB


In [74]:
# manually change VISCODE2 of some subjects
ptau.loc[(ptau['VISCODE'] == 'nv') & (ptau['RID'] == 548), 'VISCODE2'] = 'm48'
ptau.loc[(ptau['VISCODE'] == 'nv') & (ptau['RID'] == 1326), 'VISCODE2'] = 'm48'
ptau.loc[(ptau['VISCODE'] == 'nv') & (ptau['RID'] == 4499), 'VISCODE2'] = 'm48'
ptau.loc[(ptau['VISCODE'] == 'nv') & (ptau['RID'] == 4660), 'VISCODE2'] = 'm12'
ptau.loc[(ptau['VISCODE'] == 'sc') & (ptau['RID'] == 113), 'VISCODE2'] = 'm60'


In [75]:
## keep only rows with viscode2 and ptau data, where viscode2 != 'nv'
ptau2=ptau.loc[(pd.notnull(ptau['VISCODE2'])) & (pd.notnull(ptau['PLASMAPTAU181'])) & (ptau['VISCODE2'] != 'nv')]
ptau2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3756 entries, 0 to 3757
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RID            3756 non-null   int64  
 1   VISCODE        3756 non-null   object 
 2   VISCODE2       3756 non-null   object 
 3   EXAMDATE       3756 non-null   object 
 4   VID            3756 non-null   int64  
 5   PLASMAPTAU181  3756 non-null   float64
 6   COMMENT        6 non-null      object 
 7   update_stamp   3756 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 264.1+ KB


In [76]:
print(f"there are {len(ptau2[ptau2.duplicated(subset=['RID','VISCODE2'],keep=False)])} duplicates")

there are 108 duplicates


In [77]:
ptau2.drop_duplicates(subset=['RID','VISCODE2'],keep='first',inplace=True)

In [78]:
ptausm=ptau2[['RID','VISCODE2', 'PLASMAPTAU181','EXAMDATE']]
ptausm.rename(columns={'EXAMDATE':'PTAU_EXAMDATE'},inplace=True)

In [79]:
merge_ptau=merge_plasma.merge(ptausm,on=['RID','VISCODE2'],how='left')
print(f"Filled in {len(merge_ptau.loc[pd.notnull(merge_ptau['PLASMAPTAU181'])])} values")

Filled in 3700 values


## Step 9: MMSE: check data

In [80]:
mmse=pd.read_csv("ADNIspreadsheets/Cognition/MMSE_30May2024.csv")
len(mmse)

13303

In [81]:
print(f"Viscode2 null values: {len(mmse.loc[pd.isnull(mmse['VISCODE2'])])}")
print(f"MMSCORE null values: {len(mmse.loc[pd.isnull(mmse['MMSCORE'])])}")

Viscode2 null values: 262
MMSCORE null values: 105


In [82]:
# Remove rows missing viscode 2 or MMSCORE (including -1 values), and Viscode2 is not "f"
mmse2=mmse.loc[(pd.notnull(mmse['VISCODE2'])) & (pd.notnull(mmse['MMSCORE'])) &
(mmse['VISCODE2'] != "f") & (mmse['MMSCORE'] >= 0)]
print(len(mmse2))

12553


In [83]:
## Check for duplicates: 
mmse2[mmse2.duplicated(subset=['RID','VISCODE2'],keep=False)]

,PHASE,PTID,RID,VISCODE,VISCODE2,VISDATE,DONE,NDREASON,SOURCE,DATE,...,MMDRAW,MMSCORE,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
4927,ADNIGO,031_S_0830,830,m60,m60,NaN,NaN,NaN,NaN,NaN,...,1.0,25.0,1352,21,2011-09-23,2014-07-08,NaN,NaN,NaN,2014-07-08 19:02:06.0
5312,ADNI2,031_S_0830,830,v06,m60,2011-09-22,NaN,NaN,NaN,NaN,...,1.0,23.0,668,21,2011-09-23,2012-07-19,NaN,NaN,NaN,2012-11-16 12:25:07.0


In [84]:
# drop duplicate where RID == 830 and VISCODE==m60
mmse2=mmse2.drop([mmse2.loc[(mmse2['RID'] == 830) & (mmse2['VISCODE'] == 'm60')].index.values[0]])
print(len(mmse2[mmse2.duplicated(subset=['RID','VISCODE2'],keep=False)]))

0


In [85]:
# replace sc with bl
for index,row in mmse2.iterrows():
    if row['VISCODE2'] == 'sc':
        mmse2.at[index,'VISCODE2'] = "bl"
#check for dupes again
print(len(mmse2[mmse2.duplicated(subset=['RID','VISCODE2'],keep=False)]))

0


In [86]:
## select only columns to merge
mmsesm=mmse2[['RID','VISCODE2','VISDATE', 'MMSCORE']]
mmsesm.rename(columns={'VISDATE':'EXAMDATE_MMSE', "MMSCORE": "MMSE_validate"},inplace=True)

In [87]:
merge_mmse=merge_ptau.merge(mmsesm,on=['RID','VISCODE2'],how='left')
len(merge_mmse)
print(f"Gained {len(merge_mmse.loc[(pd.isnull(merge_mmse['MMSE'])) & (pd.notnull(merge_mmse['MMSE_validate']))])} data points.")
print(f"Still have {len(merge_mmse.loc[(pd.isnull(merge_mmse['MMSE'])) & (pd.isnull(merge_mmse['MMSE_validate']))])} null values.")

Gained 17 data points.
Still have 4931 null values.


In [88]:
## Add any new _validate values to original column
new=0
for index,row in merge_mmse.iterrows():
    if pd.isnull(row['MMSE']) and pd.notnull(row['MMSE_validate']):
        merge_mmse.at[index,'MMSE'] = row['MMSE_validate']
        new +=1
        
print(f"Extrapolated {new} data points to original column")
print()

## Check for any data points that are different both columns
nomatch=len(merge_mmse.loc[(pd.notnull(merge_mmse['MMSE'])) & (pd.notnull(merge_mmse["MMSE_validate"]))
& (merge_mmse['MMSE'] != merge_mmse['MMSE_validate'])])  ## if both are not null and value does not match


## If 0 columns different, drop validate column
if nomatch == 0:
    print('All values the same between original column and validate column, dropping validate column.')
    merge_mmse.drop(columns=['MMSE_validate'],inplace=True)
else:
    print('Some values different between original column and validate column, keeping validate column.')

print()
print(merge_mmse.info())

Extrapolated 17 data points to original column

Some values different between original column and validate column, keeping validate column.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 126 entries, RID to MMSE_validate
dtypes: datetime64[ns](2), float64(95), int64(3), object(26)
memory usage: 15.8+ MB
None


In [89]:
nomatch=merge_mmse.loc[(pd.notnull(merge_mmse['MMSE'])) & (pd.notnull(merge_mmse["MMSE_validate"]))
& (merge_mmse['MMSE'] != merge_mmse['MMSE_validate'])]
nomatchsm=nomatch[['RID','VISCODE2','PHASE_MERGE','EXAMDATE_MERGE','MMSE','MMSE_validate']]
len(nomatchsm)

173

In [90]:
nomatchsm['MMSE_diff'] = nomatchsm['MMSE'] - nomatchsm['MMSE_validate']
nomatchsm['MMSE_diff'].describe()

count    173.000000
mean      -0.317919
std        2.312145
min       -9.000000
25%       -1.000000
50%        1.000000
75%        1.000000
max        6.000000
Name: MMSE_diff, dtype: float64

In [91]:
nomatchsm['PHASE_MERGE'].value_counts()

PHASE_MERGE
ADNI3    172
ADNI2      1
Name: count, dtype: int64

In [92]:
nomatchsm['VISCODE2'].value_counts()

VISCODE2
bl      33
m12     25
m24     18
m84      8
m48      8
m96      7
m30      7
m114     7
m102     6
m90      6
m60      5
m72      5
m132     5
m126     4
m150     3
m66      3
m174     3
m120     3
m36      3
m138     3
m108     3
m42      2
m54      2
m78      1
m162     1
m18      1
m186     1
Name: count, dtype: int64

## Step 10: Neurobat / LDELTOTAL: check data

In [93]:
neuro=pd.read_csv("ADNIspreadsheets/Cognition/NEUROBAT_31May2024.csv")
print(len(neuro))
neuro.head()

15514


/var/folders/p8/czc1tc6x0t9d8stjlfnm009c0000gn/T/ipykernel_17253/607201030.py:1: DtypeWarning: Columns (10,82) have mixed types. Specify dtype option on import or set low_memory=False.
  neuro=pd.read_csv("ADNIspreadsheets/Cognition/NEUROBAT_31May2024.csv")


,Phase,ID,PTID,RID,SITEID,VISCODE,VISCODE2,VISDATE,USERDATE,USERDATE2,...,MINTSEMCUE,MINTTOTAL,MINTUNCUED,RAVLT_forgetting,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,DATE,SOURCE,update_stamp
0,ADNI1,12,011_S_0002,2,107,sc,sc,2005-08-17,2005-08-17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
1,ADNI1,14,011_S_0003,3,107,sc,sc,2005-08-18,2005-08-18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
2,ADNI1,16,022_S_0004,4,10,sc,sc,2005-08-18,2005-08-18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
3,ADNI1,18,011_S_0005,5,107,sc,sc,2005-08-23,2005-08-23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0
4,ADNI1,20,022_S_0007,7,10,sc,sc,2005-08-25,2005-08-25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-26 16:22:44.0


In [94]:
print(f"Viscode2 null values: {len(neuro.loc[pd.isnull(neuro['VISCODE2'])])}")
print(f"LDELTOTAL null values: {len(neuro.loc[pd.isnull(neuro['LDELTOTAL'])])}")

Viscode2 null values: 17
LDELTOTAL null values: 4520


In [95]:
# Remove rows missing viscode 2 or LDEDTOTAL (including -1 values), and Viscode2 is not "f"
neuro2=neuro.loc[(pd.notnull(neuro['VISCODE2'])) & (pd.notnull(neuro['LDELTOTAL'])) &
(neuro['VISCODE2'] != "f") & (neuro['VISCODE2'] != "uns1") & (neuro['LDELTOTAL'] >= 0)]
print(len(neuro2))  

10545


In [96]:
## Check for duplicates: 
neuro2[neuro2.duplicated(subset=['RID','VISCODE2'],keep=False)]

,Phase,ID,PTID,RID,SITEID,VISCODE,VISCODE2,VISDATE,USERDATE,USERDATE2,...,MINTSEMCUE,MINTTOTAL,MINTUNCUED,RAVLT_forgetting,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,DATE,SOURCE,update_stamp
5902,ADNIGO,1630,031_S_0830,830,21,m60,m60,NaN,2011-09-23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-16 09:42:20.0
6401,ADNI2,882,031_S_0830,830,21,v06,m60,2011-09-22,2011-09-23,2014-12-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-18 19:18:43.0


In [97]:
# drop duplicate where RID == 830 and VISCODE==m60
neuro2=neuro2.drop([neuro2.loc[(neuro2['RID'] == 830) & (neuro2['VISCODE'] == 'm60')].index.values[0]])

print(len(neuro2[neuro2.duplicated(subset=['RID','VISCODE2'],keep=False)]))

0


In [98]:
# replace sc with bl
for index,row in neuro2.iterrows():
    if row['VISCODE2'] == 'sc':
        neuro2.at[index,'VISCODE2'] = "bl"
#check for dupes again
print(len(neuro2[neuro2.duplicated(subset=['RID','VISCODE2'],keep=False)]))

0


In [99]:
neurosm=neuro2[['RID','VISCODE2','VISDATE', 'LDELTOTAL']]
neurosm.rename(columns={'VISDATE':'EXAMDATE_LDELTOTAL',"LDELTOTAL":"LDELTOTAL_validate"},inplace=True)
len(neurosm)

10544

In [100]:
merge_ldel=merge_mmse.merge(neurosm,on=['RID','VISCODE2'],how='left')
print(len(merge_ldel))
print(f"Gained {len(merge_ldel.loc[(pd.isnull(merge_ldel['LDELTOTAL'])) & (pd.notnull(merge_ldel['LDELTOTAL_validate']))])} data points.")
print(f"Still have {len(merge_ldel.loc[(pd.isnull(merge_ldel['LDELTOTAL'])) & (pd.isnull(merge_ldel['LDELTOTAL_validate']))])} null values.")

16417
Gained 7 data points.
Still have 6968 null values.


In [101]:
## Add any new _validate values to original column
new=0
for index,row in merge_ldel.iterrows():
    if pd.isnull(row['LDELTOTAL']) and pd.notnull(row['LDELTOTAL_validate']):
        merge_ldel.at[index,'LDELTOTAL'] = row['LDELTOTAL_validate']
        new +=1
        
print(f"Extrapolated {new} data points to original column")
print()

## Check for any data points that are different both columns
nomatch=len(merge_ldel.loc[(pd.notnull(merge_ldel['LDELTOTAL'])) & (pd.notnull(merge_ldel["LDELTOTAL_validate"]))
& (merge_ldel['LDELTOTAL'] != merge_ldel['LDELTOTAL_validate'])])  ## if both are not null and value does not match


## If 0 columns different, drop validate column
if nomatch == 0:
    print('All values the same between original column and validate column, dropping validate column.')
    merge_ldel.drop(columns=['LDELTOTAL_validate'],inplace=True)
else:
    print('Some values different between original column and validate column, keeping validate column.')

print()
print(merge_ldel.info())

Extrapolated 7 data points to original column

All values the same between original column and validate column, dropping validate column.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 127 entries, RID to EXAMDATE_LDELTOTAL
dtypes: datetime64[ns](2), float64(95), int64(3), object(27)
memory usage: 15.9+ MB
None


## Step 11: Imputed Amyloid Status and Diagnosis: add data

In [102]:
print(f"still missing DX: {len(merge_ldel.loc[pd.isnull(merge_ldel['DX'])])}")
print(f"still missing Amyloid Status: {len(merge_ldel.loc[pd.isnull(merge_ldel['AMYLOID_STATUS'])])}")

still missing DX: 4925
still missing Amyloid Status: 12767


In [103]:
merge_imp = merge_ldel.copy()

In [104]:
## Make copies of DX and EXAMDATE_MERGE column to use in making backwards, forwards, and nearest columns
merge_imp['DX_copy'] = merge_imp['DX']
merge_imp['EXAMDATE_copy'] = merge_imp['EXAMDATE_MERGE']
merge_imp['AMYLOID_STATUS_copy'] = merge_imp['AMYLOID_STATUS']

### Fill NA for Amyloid Status

In [105]:
## Make column filling from previous tp
amystat_backward = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                       # cols to match on
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','AMYLOID_STATUS_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','AMYLOID_STATUS_copy']), # cols to match on + cols to add
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='backward',
    ).rename(columns={'AMYLOID_STATUS_copy':'AMYLOID_STATUS_fillna_previous','EXAMDATE_copy':'AMYLOID_STATUS_fillna_previous_EXAMDATE'}).sort_values('RID')

amystat_backward.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   RID                                      16417 non-null  int64         
 1   EXAMDATE_MERGE                           16417 non-null  datetime64[ns]
 2   AMYLOID_STATUS_fillna_previous           9878 non-null   float64       
 3   AMYLOID_STATUS_fillna_previous_EXAMDATE  9878 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 641.3 KB


In [106]:
## Make column filling from forward tp
amystat_forward = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                       # cols to match on
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','AMYLOID_STATUS_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','AMYLOID_STATUS_copy']), # cols to match on + cols to add
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='forward',
    ).rename(columns={'AMYLOID_STATUS_copy':'AMYLOID_STATUS_fillna_forward','EXAMDATE_copy':'AMYLOID_STATUS_fillna_forward_EXAMDATE'}).sort_values('RID')

amystat_forward.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   RID                                     16417 non-null  int64         
 1   EXAMDATE_MERGE                          16417 non-null  datetime64[ns]
 2   AMYLOID_STATUS_fillna_forward           9212 non-null   float64       
 3   AMYLOID_STATUS_fillna_forward_EXAMDATE  9212 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 641.3 KB


In [107]:
## Make column filling from nearest tp
amystat_nearest = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                       # cols to match on
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','AMYLOID_STATUS_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','AMYLOID_STATUS_copy']), # cols to match on + cols to add
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='nearest'
    ).rename(columns={'AMYLOID_STATUS_copy':'AMYLOID_STATUS_fillna_nearest','EXAMDATE_copy':'AMYLOID_STATUS_fillna_nearest_EXAMDATE'}).sort_values('RID')

amystat_nearest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   RID                                     16417 non-null  int64         
 1   EXAMDATE_MERGE                          16417 non-null  datetime64[ns]
 2   AMYLOID_STATUS_fillna_nearest           11827 non-null  float64       
 3   AMYLOID_STATUS_fillna_nearest_EXAMDATE  11827 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 641.3 KB


In [108]:
## How many subjects missing an Amyloid Status?
temp=amystat_nearest.loc[pd.isnull(amystat_nearest['AMYLOID_STATUS_fillna_nearest'])]
print(len(temp))
print(len(temp['RID'].unique()))

## Fewer missing than in old sheet (1406 subjects w/o amyloid status)

4590
739


In [109]:
## Add each to main df and add timedelta column
merge_imp=merge_imp.merge(amystat_backward, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['AMYLOID_STATUS_fillna_previous_timedelta'] = (merge_imp['AMYLOID_STATUS_fillna_previous_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp=merge_imp.merge(amystat_forward, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['AMYLOID_STATUS_fillna_forward_timedelta'] = (merge_imp['AMYLOID_STATUS_fillna_forward_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp=merge_imp.merge(amystat_nearest, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['AMYLOID_STATUS_fillna_nearest_timedelta'] = (merge_imp['AMYLOID_STATUS_fillna_nearest_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 139 entries, RID to AMYLOID_STATUS_fillna_nearest_timedelta
dtypes: datetime64[ns](6), float64(102), int64(3), object(28)
memory usage: 17.4+ MB


In [110]:
merge_imp.value_counts(['AMYLOID_STATUS','AMYLOID_STATUS_fillna_previous'],dropna=False)

AMYLOID_STATUS  AMYLOID_STATUS_fillna_previous
NaN             NaN                               6539
                0.0                               3133
                1.0                               3095
0.0             0.0                               1944
1.0             1.0                               1706
Name: count, dtype: int64

In [111]:
merge_imp.value_counts(['AMYLOID_STATUS','AMYLOID_STATUS_fillna_forward'],dropna=False)

AMYLOID_STATUS  AMYLOID_STATUS_fillna_forward
NaN             NaN                              7205
                0.0                              2852
                1.0                              2710
0.0             0.0                              1944
1.0             1.0                              1706
Name: count, dtype: int64

In [112]:
merge_imp.value_counts(['AMYLOID_STATUS','AMYLOID_STATUS_fillna_nearest'],dropna=False)

AMYLOID_STATUS  AMYLOID_STATUS_fillna_nearest
NaN             NaN                              4590
                1.0                              4140
                0.0                              4037
0.0             0.0                              1944
1.0             1.0                              1706
Name: count, dtype: int64

In [113]:
# how many are over 3 years away?
len(merge_imp.loc[merge_imp['AMYLOID_STATUS_fillna_nearest_timedelta'] > 1095])
## could add ", tolerance=pd.Timedelta('1095d')" to filter those out

1109

### Fill na for DX column

In [114]:
## Make DX_previous column
Diagnosis_backward = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                       # cols to match on
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','DX_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','DX_copy']), # cols to match on + cols to add
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='backward',
    ).rename(columns={'DX_copy':'DX_fillna_previous','EXAMDATE_copy':'DX_fillna_previous_EXAMDATE'}).sort_values('RID')

Diagnosis_backward.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   RID                          16417 non-null  int64         
 1   EXAMDATE_MERGE               16417 non-null  datetime64[ns]
 2   DX_fillna_previous           16417 non-null  object        
 3   DX_fillna_previous_EXAMDATE  16417 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 641.3+ KB


In [115]:
## Make forward DX cols
Diagnosis_forward = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                            
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','DX_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','DX_copy']),
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='forward',
    ).rename(columns={'DX_copy':'DX_fillna_forward','EXAMDATE_copy':'DX_fillna_forward_EXAMDATE'}).sort_values('RID')

Diagnosis_forward.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   RID                         16417 non-null  int64         
 1   EXAMDATE_MERGE              16417 non-null  datetime64[ns]
 2   DX_fillna_forward           15173 non-null  object        
 3   DX_fillna_forward_EXAMDATE  15173 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 641.3+ KB


In [116]:
## Create nearest col
Diagnosis_nearest = pd.merge_asof(
    left=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE']].dropna(subset=['EXAMDATE_MERGE']).reset_index(drop=True),                            
    right=merge_imp.sort_values('EXAMDATE_MERGE')[['RID','EXAMDATE_MERGE','DX_copy','EXAMDATE_copy']].dropna(axis=0, subset=['EXAMDATE_MERGE','DX_copy']),
    by=['RID'],
    allow_exact_matches=True,
    on='EXAMDATE_MERGE', direction='nearest',
    ).rename(columns={'DX_copy':'DX_fillna_nearest','EXAMDATE_copy':'DX_fillna_nearest_EXAMDATE'}).sort_values('RID')

Diagnosis_nearest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16417 entries, 12690 to 16404
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   RID                         16417 non-null  int64         
 1   EXAMDATE_MERGE              16417 non-null  datetime64[ns]
 2   DX_fillna_nearest           16417 non-null  object        
 3   DX_fillna_nearest_EXAMDATE  16417 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 641.3+ KB


In [117]:
## Add columns to main df, add timedelta column
merge_imp=merge_imp.merge(Diagnosis_backward, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['DX_fillna_previous_timedelta'] = (merge_imp['DX_fillna_previous_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp=merge_imp.merge(Diagnosis_forward, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['DX_fillna_forward_timedelta'] = (merge_imp['DX_fillna_forward_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp=merge_imp.merge(Diagnosis_nearest, on=['RID','EXAMDATE_MERGE'],how='left')
merge_imp['DX_fillna_nearest_timedelta'] = (merge_imp['DX_fillna_nearest_EXAMDATE'] - merge_imp['EXAMDATE_MERGE']) / np.timedelta64(1,'D')

merge_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 148 entries, RID to DX_fillna_nearest_timedelta
dtypes: datetime64[ns](9), float64(105), int64(3), object(31)
memory usage: 18.5+ MB


In [118]:
merge_imp.value_counts(['DX','DX_fillna_previous'],dropna=False)

DX   DX_fillna_previous
2.0  2.0                   5004
1.0  1.0                   4032
3.0  3.0                   2456
NaN  1.0                   2218
     2.0                   1631
     3.0                   1076
Name: count, dtype: int64

In [119]:
merge_imp.value_counts(['DX','DX_fillna_forward'],dropna=False)

DX   DX_fillna_forward
2.0  2.0                  5004
1.0  1.0                  4032
3.0  3.0                  2456
NaN  1.0                  1631
     2.0                  1299
     NaN                  1244
     3.0                   751
Name: count, dtype: int64

In [120]:
merge_imp.value_counts(['DX','DX_fillna_nearest'],dropna=False)

DX   DX_fillna_nearest
2.0  2.0                  5004
1.0  1.0                  4032
3.0  3.0                  2456
NaN  1.0                  2197
     2.0                  1587
     3.0                  1141
Name: count, dtype: int64

In [121]:
## Drop *_copy columns
merge_imp = merge_imp.drop(columns = ['DX_copy','EXAMDATE_copy','AMYLOID_STATUS_copy']) 

### Boolean columns indicating if dx/status is imputed

In [122]:
bool_df = merge_imp.copy()

In [123]:
newcols=[col.replace('timedelta','imputed') for col in merge_imp.columns if 'timedelta' in col]
for col in newcols:
    bool_df[col] = None 

In [124]:
for index,row in bool_df.iterrows():
    for column in bool_df.columns:
        if "timedelta" in column:
            collist=column.split("_")
            collist.remove('timedelta')
            collist.append('imputed')
            newcol="_".join(collist)
            # print(newcol)
            if pd.notnull(row[column]):
                # print(row[column])
                if row[column] == 0:
                    # print('not imputed')
                    bool_df.at[index,newcol] = 0
                else:
                    # print('imputed')
                    bool_df.at[index,newcol] = 1
                
bool_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16417 entries, 0 to 16416
Columns: 151 entries, RID to DX_fillna_nearest_imputed
dtypes: datetime64[ns](8), float64(104), int64(3), object(36)
memory usage: 18.9+ MB


### DX change column

In [125]:
## sort and reset index so the index-1 can be used to compare to previous DX value
dxchangedf=bool_df.sort_values(by=['RID','EXAMDATE_MERGE'])
dxchangedf.reset_index(drop=True,inplace=True)

In [126]:
## DX change uses nearest fillna column
allsubs = dxchangedf['RID'].unique().tolist()
dxnear_index = dxchangedf.columns.get_loc('DX_fillna_nearest') 
dxchangedf['DX_changed_fillna_nearest'] = None

for sub in allsubs:
    match=dxchangedf.loc[dxchangedf['RID'] == sub]
    match_index_list = match.index.tolist()
    for i in match_index_list:
        if i > match_index_list[0]: ## not the first entry
            if dxchangedf.iloc[i, dxnear_index] == dxchangedf.iloc[i-1, dxnear_index]:
                # print('same dx')
                dxchangedf.at[i,'DX_changed_fillna_nearest'] = 0
            else:
                # print('dx has changed')
                # print(f"This tp: {dxchangedf.iloc[i, dxnear_index]}, previous tp: {dxchangedf.iloc[i-1, dxnear_index]}")
                dxchangedf.at[i,'DX_changed_fillna_nearest'] = 1


dxchangedf['DX_changed_fillna_nearest'].value_counts(dropna=False)
## 2430 null == 2430 subjects' first entries 

DX_changed_fillna_nearest
0       13202
None     2430
1         785
Name: count, dtype: int64

## Step 12: Final NaN counts and export

In [127]:
## check for duplicates
merge_impamy=dxchangedf.copy()
dupes=merge_impamy[merge_impamy.duplicated(subset=["RID", 'VISCODE2'],keep=False)]
if len(dupes) > 0:
    print('WARNING:delete duplicate sessions')
else:
    print('INFO:no duplicates')

INFO:no duplicates


In [128]:
print("DX column:")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['DX'])])} null values in original adnimerge csv.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['DX'])])} null values after checking.")
print(f"Gained {len(adnimerge.loc[pd.isnull(adnimerge['DX'])]) - len(merge_impamy.loc[pd.isnull(merge_impamy['DX'])])} rows of data.")

DX column:
4959 null values in original adnimerge csv.
4925 null values after checking.
Gained 34 rows of data.


In [129]:
print("Birthdate column:")
print(f"Added {len(merge_impamy.loc[pd.notnull(merge_impamy['PTBIRTHDATE'])])} rows of data.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['PTBIRTHDATE'])])} rows have null values.")

Birthdate column:
Added 16417 rows of data.
0 rows have null values.


In [130]:
print("CDRSB column:")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['CDRSB'])])} null values in original adnimerge csv.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['CDRSB'])])} null values after checking.")
print(f"Gained {len(adnimerge.loc[pd.isnull(adnimerge['CDRSB'])]) - len(merge_impamy.loc[pd.isnull(merge_impamy['CDRSB'])])} rows of data.")

CDRSB column:
4670 null values in original adnimerge csv.
4668 null values after checking.
Gained 2 rows of data.


In [131]:
print("Amyloid Status column:")
print(f"Added {len(merge_impamy.loc[pd.notnull(merge_impamy['AMYLOID_STATUS'])])} rows of data.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['AMYLOID_STATUS'])])} rows have null values.")

Amyloid Status column:
Added 3650 rows of data.
12767 rows have null values.


In [132]:
print("APOE4 column:")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['APOE4'])])} null values in original adnimerge csv.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['APOE4'])])} null values after checking.")
print(f"Gained {len(adnimerge.loc[pd.isnull(adnimerge['APOE4'])]) - len(merge_impamy.loc[pd.isnull(merge_impamy['APOE4'])])} rows of data.")

APOE4 column:
365 null values in original adnimerge csv.
48 null values after checking.
Gained 317 rows of data.


In [133]:
print("PLASMA_NFL column:")
print(f"Added {len(merge_impamy.loc[pd.notnull(merge_impamy['PLASMA_NFL'])])} rows of data.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['PLASMA_NFL'])])} rows have null values.")

PLASMA_NFL column:
Added 3701 rows of data.
12716 rows have null values.


In [134]:
print("PTAU column:")
print(f"Added {len(merge_impamy.loc[pd.notnull(merge_impamy['PTAU'])])} rows of data.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['PTAU'])])} rows have null values.")

PTAU column:
Added 2369 rows of data.
14048 rows have null values.


In [135]:
print("MMSE column:")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['MMSE'])])} null values in original adnimerge csv.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['MMSE'])])} null values after checking.")
print(f"Gained {len(adnimerge.loc[pd.isnull(adnimerge['MMSE'])]) - len(merge_impamy.loc[pd.isnull(merge_impamy['MMSE'])])} rows of data.")

MMSE column:
4948 null values in original adnimerge csv.
4931 null values after checking.
Gained 17 rows of data.


In [136]:
print("LDELTOTAL column:")
print(f"{len(adnimerge.loc[pd.isnull(adnimerge['LDELTOTAL'])])} null values in original adnimerge csv.")
print(f"{len(merge_impamy.loc[pd.isnull(merge_impamy['LDELTOTAL'])])} null values after checking.")
print(f"Gained {len(adnimerge.loc[pd.isnull(adnimerge['LDELTOTAL'])]) - len(merge_impamy.loc[pd.isnull(merge_impamy['LDELTOTAL'])])} rows of data.")

LDELTOTAL column:
6975 null values in original adnimerge csv.
6968 null values after checking.
Gained 7 rows of data.


In [137]:
current_date = datetime.now().strftime("%Y_%m_%d")
merge_impamy.to_csv(f"/project/wolk/ADNI2018/analysis_output/data/ADNIMERGE_PICSLversion_{current_date}.csv",index=False,header=True)